In [ ]:
from math import log2
from random import randint

In [ ]:
dataset = [
    [1,1,1,1],
    [0,0,1,1],
    [0,1,0,0],
    [1,0,1,0],
    [1,1,1,1],
    [1,1,0,1],
    [0,0,0,0],
    [1,1,0,1],
    [0,1,0,0],
    [0,1,0,0]
]
dataLabels = ['Ear Shape','Face Shape','Whiskers','Cat']
dataDesc = [['Pointy','Floppy'],['Round','Not Round'],['Present','Absent'],['Cat','Not Cat']]

In [ ]:
def H(c):
    d = 1 - c
    if c == 0 or c == 1:
        return 0
    return (-c * log2(c)) + (-d * log2(d))

In [ ]:
def redH(s1,s2,t1,t2):
    wh1 = (s1/(s1+s2))*(H(t1/s1) if s1!=0 else 0)
    wh2 = (s2/(s1+s2))*(H(t2/s2) if s2!=0 else 0)
    wh = H((t1+t2)/(s1+s2))-(wh1+wh2)
    return wh

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.split = -1
        self.splitNotTrue = None
        self.splitTrue = None
        self.parent = None
        self.trueCases = len([0 for i in self.data if i[3]==1])
        self.totalCases = len(self.data)
        if self.totalCases == 0: 
            self.entropy = 0
        else:
            self.entropy = H(self.trueCases/self.totalCases)
    
    def setSplit(self, split):
        self.split = split
        if split != -1:
            isNotTrue = []
            isTrue = []
            for i in self.data:
                if i[split] == 0:
                    isNotTrue.append(i)
                else:
                    isTrue.append(i)
            self.splitNotTrue = Node(isNotTrue)
            self.splitTrue = Node(isTrue)
            self.splitNotTrue.parent = self
            self.splitTrue.parent = self
        else:
            self.splitTrue = None
            self.splitNotTrue = None
            
    def __repr__(self):
        return f"\n\nData: {self.data}\n Split At: {self.split}\n Entropy: {self.entropy}\n\n"
    
    def toString(self):
        return f"Length: {(self.data)}, Split At: {self.split}, Entropy: {round(self.entropy,3)}\t"

In [ ]:
class Tree:
    def __init__(self, data):
        self.data = data
        self.tree = []
    
    def ComputationalTree(self):
        self.tree = []
        for i in self.data:
            if len(i) != 0:
                self.tree.append([])
                for j in i:
                    self.tree[-1].append(j.split)
        return self.tree
    
    def check(self, testCase):
        self.ComputationalTree()
        currCheck = 0
        prevRes = 0
        for i in self.tree:
            try:
                if i[currCheck] == -1:
                    return prevRes
                prevRes = testCase[i[currCheck]]
                currCheck = currCheck*2+prevRes
            except:
                return prevRes
    
    def __repr__(self):
        s = ""
        for i in self.data:
            s+="\n\n"
            for j in i:
                if j != None:
                    if len(j.data) != 0:
                        s+=j.toString()
        return s

In [ ]:
def BuildTree(dataset):
    rootNode = Node(dataset)
    layers = [[rootNode]]
    while True:
        bottomlayer = layers[-1]
        layers.append([])
        flag = 1
        for i in bottomlayer:
            if i != None:
                if i.entropy != 0:
                    flag = 0
        if flag == 1:
            break
        for i in bottomlayer:
            if i.entropy != 0:
                s = -1
                minh = 0
                for j in range(len(dataset[0])-1):
                    if i.parent != None:
                        i.setSplit(j)
                        h = redH(i.splitNotTrue.totalCases, i.splitTrue.totalCases, i.splitNotTrue.trueCases, i.splitTrue.trueCases)
                        if h > minh:
                            s = j
                            minh = h
                        i.setSplit(-1)
                    else:
                        i.setSplit(j)
                        h = redH(i.splitNotTrue.totalCases, i.splitTrue.totalCases, i.splitNotTrue.trueCases, i.splitTrue.trueCases)
                        if h > minh:
                            s = j
                            minh = h
                        i.setSplit(-1)
                i.setSplit(s)
                layers[-1].append(i.splitNotTrue)
                layers[-1].append(i.splitTrue)
            else:
                layers[-1].append(i)
    d = Tree(layers)
    return d

In [ ]:
def checkTrees(trees, test):
    f = 0
    t = 0
    for i in trees:
        r = i.check(test)
        if r == 0: 
            f += 1
        elif r == 1:
            t += 1
    return t/(t+f)

In [ ]:
trees = []
for _ in range(100):
    tempData = []
    for i in range(len(dataset)):
        tempData.append(dataset[randint(0, len(dataset)-1)])
    tree = BuildTree(tempData)
    trees.append(tree)
    
checkTrees(trees, [1,1,0])